In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
""" if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu" """
device ="cpu"

In [40]:
data1=pd.read_csv("./data/au1goodtr.csv")
data2=pd.read_csv("./data/au1badtr.csv")

data_train = pd.concat([data1,data2])
data_train.head(200)


,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,174.976578,-197.425861,-375.257134,365.747757,6.404815,-135.886073,0,0,-124.680233,good
1,190.675697,-201.006010,-379.607291,366.770668,27.453504,-102.730300,0,0,-174.907204,good
2,200.726700,-183.024673,-391.389656,349.262772,47.981586,-71.945885,0,0,-179.939768,good
3,192.053795,-186.327610,-348.735988,335.199928,40.310769,-72.427084,0,0,-168.923206,good
4,156.627769,-231.544955,-745.391550,385.714226,-66.977506,-365.926909,0,0,-373.551550,good
...,...,...,...,...,...,...,...,...,...,...
195,195.887184,-249.757626,-580.979090,447.969780,-79.961414,-116.353390,0,0,-115.965060,good
196,298.356209,-195.283356,-847.656755,492.402763,13.169947,-280.529323,0,0,-362.596035,good
197,232.235298,-285.036678,-1138.101110,544.236259,-128.187919,-519.133530,0,0,-543.547726,good
198,230.867767,-276.795194,-612.068682,499.696198,-105.575480,-133.584759,0,0,-175.027721,good


In [41]:

data3=pd.read_csv("./data/au1goodval.csv")
data4=pd.read_csv("./data/au1badval.csv")

data_test = pd.concat([data3,data4])
data_test.head(200)

,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,208.898582,-187.526751,-547.123218,375.724525,7.058458,-148.510716,0,0,-251.841209,good
1,118.933334,-252.201859,-479.533653,329.510193,-101.512620,-149.736131,0,0,-144.182918,good
2,117.265511,-149.988356,-490.028257,342.409821,24.276094,-108.588631,0,0,-185.394609,good
3,150.576973,-140.970683,-524.006739,349.723015,31.197696,-118.590996,0,0,-226.991315,good
4,225.162468,-126.615801,-603.233399,354.925804,79.894166,-141.575124,0,0,-259.171836,good
...,...,...,...,...,...,...,...,...,...,...
57,275.432587,-14.059968,-1434.110641,549.485474,100.477266,-911.212234,0,0,-835.456696,bad
58,269.352913,16.651454,-944.789371,462.934837,154.816675,-414.590034,0,0,-495.794320,bad
59,248.090210,10.770979,-965.339470,467.030487,4.643312,-377.550573,0,0,-470.639963,bad
60,210.551682,6.902804,-948.360529,459.999847,0.949116,-381.737738,0,0,-397.617230,bad


In [42]:
data = pd.concat([data_train,data_test])
data = data.drop(columns=["Right Shoulder X","Right Shoulder Y"])

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import torch.utils
from pickle import dump
encoder = LabelEncoder()
data["Label"] = encoder.fit_transform(data["Label"])
X=data.drop(columns="Label").to_numpy(dtype=np.float32)
y=data["Label"].to_numpy(dtype=np.float32)
#scaler=MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

train_dataset= torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))

test_dataset= torch.utils.data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
#dump(scaler, open('scaler.pkl', 'wb'))

train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=3,num_workers=3)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=3)

In [44]:

class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.LayerNorm(7),
            nn.Linear(7,36),
            nn.ReLU(),
            nn.Linear(36,15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(15,12),
            nn.ReLU(),
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,3),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    @torch.compile(fullgraph=True, dynamic=True, options = {"epilogue_fusion": True, "max_autotune": True,
                    "shape_padding": True})
    def forward(self,x):
        return self.layers(x)


In [45]:
model = slouch_detection().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Assuming you're using Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate as needed


In [46]:
total=0
for params in model.parameters():
    total+=params.numel()
total

1152

In [47]:
def train_model(model, train_loader, optimizer, criterion, epochs,device,best_acc):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            #print(outputs.size(),labels.unsqueeze(dim=1).size())
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            
            # Calculate accuracy
            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            print()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
        test_model(model, test_dataloader, criterion,device,best_acc)

# Testing loop
def test_model(model, test_loader, criterion,device,best_acc):
    model.eval()
    with torch.inference_mode():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        test_loss = running_loss / len(test_loader.dataset)
        test_acc = correct / total
        if(best_acc<test_acc):
            torch.save(model.state_dict(),"slouch_detector.pt")
            best_acc=test_acc
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [48]:
best_acc=0
train_model(model, train_dataloader, optimizer, criterion,device=device, epochs=50,best_acc=best_acc)  # Adjust epochs as needed

test_model(model, test_dataloader, criterion,device,best_acc)




























































































































































































































































































Epoch 1/50, Loss: 0.7066, Accuracy: 0.4953
Test Loss: 0.6499, Test Accuracy: 0.6435



























































































































































































































































































Epoch 2/50, Loss: 0.5791, Accuracy: 0.6863
Test Loss: 0.5709, Test Accuracy: 0.6866












































































































































































































































































In [49]:
test_model(model, test_dataloader, criterion,device,best_acc)

Test Loss: 0.4220, Test Accuracy: 0.8038
